### US Equity market performance from 1995 to 2021
In this visualization we'll look S&P 500 Index performance from 1995 to 2021. The visulization consists of two charts:
* Time Series Chart of Index prices
* Histogram of the log returns

We'll use the [Fast Interval Selector](../Interactions/Selectors.ipynb) to quickly select different time slices and perform time series analysis on the selected slice (in this case, displaying the total return and the distribution of log returns). **Click on the time series chart to activate the interval selector!**

Fun things to try:
1. `FastIntervalSelector` responds to mouse moves! Move the mouse pointer up to expand the interval, down to contract the interval. Single click toggles between freezing/unfreezing the window length. Double click totally freezes the interval selector. Move the interval selector to perform quick visual time series analysis!
2. Notice that the color of the return label changes with the sign of the total return (red for negative returns and green for positive returns)
3. What was the index performance from 1995-2005? 2000-2010? 2010-2020?
4. Does the return distribution have fat tails? (especially during 2008 financial crisis, 2020 covid crisis)

In [ ]:
import numpy as np
import pandas as pd

from ipywidgets import *
from bqplot import DateScale, LinearScale
import bqplot.pyplot as plt
from bqplot.interacts import FastIntervalSelector

In [ ]:
def regress(x, y):
    b = np.cov(x, y)[0, 1] / np.var(x)
    a = y.mean() - b * x.mean()
    return a, b

def trend_score(x, y):
    """computes trend score as a cube of correlation coeff"""
    return np.corrcoef(x, y)[0, 1] ** 3

In [ ]:
spx_prices = pd.read_csv("spx_prices.csv", index_col=0, 
                         squeeze=True, parse_dates=True)
dates = spx_prices.index
prices = spx_prices
log_returns = np.log(prices / prices.shift(1)).dropna()

In [ ]:
fig_title_tmpl = "S&P Index Prices (from {} to {})"
fig_title = fig_title_tmpl.format(dates[0].strftime("%m/%d/%Y"), 
                                  dates[-1].strftime("%m/%d/%Y"))
time_series_fig = plt.figure(title=fig_title, 
                             layout=Layout(width="1300px",
                                           height="500px"))

plt.scales(scales={"x": DateScale()})
axes_options = {"y": {"tick_format": ","}}
time_series = plt.plot(dates, prices, colors=["deepskyblue"], 
                       stroke_width=1.5, axes_options=axes_options)
return_label = plt.label([], x=[], y=[prices.max() * .9], 
                         align="middle", default_size=36, 
                         font_weight="bolder", colors=["orange"])
trend_line = plt.plot([], [])
intsel = FastIntervalSelector(scale=time_series.scales['x'], marks=[time_series])
time_series_fig.interaction = intsel

hist_fig = plt.figure(title="Histogram Of Daily Returns",
                      layout=Layout(width="900px",
                                    height="450px",
                                    margin="0px 150px 0px 150px"))
plt.scales(scales={"x": LinearScale(min=-.1, max=.1)})
hist = plt.hist(log_returns, colors=["salmon"], bins=75)
for axis in hist_fig.axes:
    axis.grid_lines = "none"
    if axis.orientation != "vertical":
        axis.tick_format = ".0%"

def update_trend_line(*args):
    selected_idx = time_series.selected
    if selected_idx is not None:
        x = np.array(selected_idx)
        y = time_series.y[selected_idx]

        a, b = regress(x, y)
        ts = trend_score(x, y)
        end_points = [selected_idx[0], selected_idx[-1]]
        pct_return = prices[end_points[1]] / prices[end_points[0]] - 1
        
        with return_label.hold_sync():
            return_label.text = ['{:.0%}'.format(pct_return)]
            return_label.x = [time_series.x[(end_points[0] + end_points[1]) // 2]]
        
        with trend_line.hold_sync():
            trend_line.x = time_series.x[end_points]
            trend_line.y = a + b * np.array(end_points)
            if np.abs(ts) < .1:
                trend_line.colors = ['yellow']
            elif ts > .1:
                trend_line.colors = ['lime']
            else:
                trend_line.colors = ['red']    
            trend_line.stroke_width = 2 + 6 * np.abs(ts)
        time_series_fig.title = fig_title_tmpl.format(*[dates[i].strftime('%m/%d/%Y') \
                                                        for i in end_points])
        # update hist with returns
        hist.sample = np.log(y[1:] / y[:-1])

time_series.observe(update_trend_line, 'selected')
VBox([time_series_fig, hist_fig])